In [1]:
!pip install evaluate > /dev/null 2>&1


In [2]:
!pip install pyarrow==8.0.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 35.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.1.0
    Uninstalling pyarrow-16.1.0:
      Successfully uninstalled pyarrow-16.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 8.0.0 which is incompatible.
datasets 2.20.0 requires pyarrow>=15.0.0, but you have pyarrow 8.0.0 which is incompatible.


In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
import transformers
from transformers import *
import numpy as np
import evaluate
from tqdm import tqdm

/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader, TensorDataset

SEED = 123

torch.manual_seed(SEED)

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [7]:
try:
    from google.colab import drive
    drive.mount('/content/drive')

    train_path = '/content/drive/My Drive/ssne/train_data.csv'
    test_path = '/content/drive/My Drive/ssne/test_data.csv'

    print('Successfully loaded from Google Drive')

except Exception:

    train_path = 'train_data.csv'
    test_path = 'test_data.csv'

    print('Failed to load from Google Drive')

Mounted at /content/drive
Successfully loaded from Google Drive


In [8]:
train_and_val_data = pd.read_csv(train_path, header=0)

test_data = pd.read_csv(test_path, header=None)
test_data.columns = ['review']  # zakładamy, że plik ma tylko kolumnę 'review'


print(test_data.head())

test_texts = test_data['review'].astype(str).tolist()


# print(train_and_val_data.head())

                                              review
0  great hotel location stayed 4 nts 24th 28th ja...
1  n't return overall disappointed hotel, no hot ...
2  great value location desired problem hotel loc...
3  kind helpfull people people kind helpful.we no...
4  absolutely fabulous melia comfortable star hot...


In [9]:
from sklearn.model_selection import train_test_split

split = 0.8

train_data, val_data = train_test_split(train_and_val_data, test_size=1 - split, random_state=SEED)
print(train_data.head())
print(val_data.head())

                                                  review  rating
10014  awful bedrooms wonderful staff booked hotel ce...       1
15618  really good value stayed meriton world towers ...       3
12711  just perfect stayed 4 nights recently husband ...       4
16144  fine base sights went hotel august bank holida...       2
9910   wonderful hotel just returned days hotel wante...       4
                                                 review  rating
7831  le walt review stayed le walt paris oct 4 oct ...       2
4448  excellent location 2 adults 1 child stayed omn...       4
5393  perfect hotel short trip hotel really star com...       4
9071  fair room quality unfriendly staff make sure r...       0
6629  n't let bed bugs bite hotel pretty ok bed bug ...       2


# Class imbalance

In [10]:
train_data_occurence_count = train_data['rating'].value_counts().to_dict()
val_data_occurence_count = val_data['rating'].value_counts().to_dict()

print((
    'Class occurence count:\n'
    'Traing data: {}\n'
    'Validation data: {}'
).format(train_data_occurence_count, val_data_occurence_count))

Class occurence count:
Traing data: {4: 5776, 3: 3880, 2: 1426, 1: 1149, 0: 882}
Validation data: {4: 1467, 3: 951, 2: 321, 1: 285, 0: 255}


In [11]:
weights = {cls : 1 / train_data_occurence_count[cls] for cls in train_data_occurence_count.keys()}
print(weights)

{4: 0.00017313019390581717, 3: 0.0002577319587628866, 2: 0.0007012622720897616, 1: 0.0008703220191470844, 0: 0.0011337868480725624}


In [12]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_text = train_data['review']
train_labels = train_data['rating']

val_text = val_data['review']
val_labels = val_data['rating']

train_embeddings = tokenizer(
    list(train_text),
    padding='max_length',
    truncation=True,
)

val_embeddings = tokenizer(
    list(val_text),
    padding='max_length',
    truncation=True
)

test_embeddings = tokenizer(
    test_texts,
    padding='max_length',
    truncation=True
)

print("test_embeddings")
print(test_embeddings)

train_input_ids = torch.tensor(train_embeddings['input_ids'])
train_attention_mask = torch.tensor(train_embeddings['attention_mask'])
train_labels = torch.tensor(train_labels.values)

val_input_ids = torch.tensor(val_embeddings['input_ids'])
val_attention_mask = torch.tensor(val_embeddings['attention_mask'])
val_labels = torch.tensor(val_labels.values)

test_input_ids = torch.tensor(test_embeddings['input_ids'])
test_attention_mask = torch.tensor(test_embeddings['attention_mask'])

train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_mask, val_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask)

train_dloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dloader = DataLoader(val_dataset, batch_size=16)
test_dloader = DataLoader(test_dataset, batch_size=16)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.41.2",
  "vocab_size": 30522
}



vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropo

test_embeddings


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
batch = next(iter(train_dloader))
print([t.shape for t in batch])

[torch.Size([16, 512]), torch.Size([16, 512]), torch.Size([16])]


In [14]:
# model = AutoModel.from_pretrained(model_name, num_labels=5)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=5)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [15]:
# from transformers import AutoModelForSequenceClassification

# model_name = 'xlnet-base-cased'
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)
# model = model.to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--xlnet-base-cased/snapshots/ceaa69c7bc5e512b5007106a7ccbb7daf24b2c79/config.json
Model config XLNetConfig {
  "_name_or_path": "xlnet-base-cased",
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--xlnet-base-cased/snapshots/ceaa69c7bc5e512b5007106a7ccbb7daf24b2c79/pytorch_model.bin
Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_pro

In [16]:
# from transformers import AlbertForSequenceClassification

# model_name = 'albert-base-v2'
# model = AlbertForSequenceClassification.from_pretrained(model_name, num_labels=5)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# model.to(device)

In [18]:
num_epochs = 10
loss_fun = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    losses = []
    for input_ids, attention_mask, labels in train_dloader:

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model.forward(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        loss = loss_fun(outputs.logits, labels)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    print(f'{np.mean(losses):.5f}')

1.36883
1.36591


KeyboardInterrupt: 

In [21]:
metric = evaluate.load("accuracy")
model.eval()
for input_ids, attention_mask, labels in val_dloader:

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model.forward(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

    predictions = torch.argmax(outputs.logits, dim=-1)
    metric.add_batch(predictions=predictions, references=labels)

print(f'Accuracy: {metric.compute()["accuracy"]}')

Accuracy: 0.4473924977127173


distilbert-base-uncased:
Accuracy: 0.623360780725831

albert-base-v2:
Accuracy: 0.4473924977127173

XLNetForSequenceClassification:
Accuracy: 0.4473924977127173

In [22]:
combined_preds = np.empty((0,))

model.eval()
for input_ids, attention_mask in test_dloader:

    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        outputs = model.forward(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

    predictions = torch.argmax(outputs.logits, dim=-1)
    preds_np = predictions.detach().cpu().numpy()
    print(preds_np)
    combined_preds = np.concatenate((combined_preds, preds_np)).astype(int)
np.savetxt('piatek_Kubiszyn_Sobiech.csv', combined_preds, delimiter=',', fmt='%d')


[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[4 4 4 4 4 4 4